In [11]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

In [3]:
file = open("data/ar-en.tmx", "r", encoding="UTF-8")
contents = file.read()

In [4]:
soup = BeautifulSoup(contents, 'xml')

In [5]:
tu = soup.find_all('tu')

In [6]:
# e.g. of arabic and english pair
    # <tu>
    #   <tuv xml:lang="ar"><seg>2) #40, على الرغم من أنه يمكن أن ينظر إلى إصلاحه بعد ذلك. </seg></tuv>
    #   <tuv xml:lang="en"><seg>2) #40, though he can be seen reforming afterward. </seg></tuv>
    # </tu>

In [10]:
# get pairs
pairs = []
for t in tqdm(tu):
    ar = t.find('tuv', {'xml:lang': 'ar'}).find('seg').text
    en = t.find('tuv', {'xml:lang': 'en'}).find('seg').text
    pairs.append((ar, en))

  0%|          | 0/999763 [00:00<?, ?it/s]

100%|██████████| 999763/999763 [00:52<00:00, 18943.63it/s]


In [12]:
df = pd.DataFrame(pairs, columns=['ar', 'en'])

In [13]:
df

,ar,en
0,إلى ربها ناظرة: أي منتظرة لرحمته ودخول جنته.,He is sometimes seen having arguments with his...
1,في البداية حققوا بعض المكاسب بما في ذلك الاستي...,"Initially, they made some gains, including cap..."
2,الأدميرال هالسي (1944) يرتدي خوذة الألياف الأم...,Admiral Halsey (1944) wearing an American fibe...
3,اطلع عليه بتاريخ 24 أبريل 2011. Yemen deal may...,Yemen deal may be done within week: officials.
4,"عن ""رب العالمين"" سبحانه وتعالى.",Opsaridium ubangiense.
...,...,...
999758,وتتم معظم هذه الأعمال في مدينة هاميلتون العاصمة.,Most of the city is in the Hamilton City Schoo...
999759,ومع تقدم الرواية يسمع جون حول مادة اسمها «الجل...,"As the novel progresses, John learns of a subs..."
999760,قامت بأداء أول سينيمائي لها عام 2002 أمام دنيس...,"She made her film debut in 2002, opposite Denn..."
999761,حزين كما الجحيم (15 قصة قصيرة).,The Great Return (1915) — short story.


In [14]:
# filter pairs where the arabic text is equal to the english text or vice versa
df = df[df['ar'] != df['en']]

In [16]:
duplicates = []
for pair in df.itertuples():
    if pair.ar == pair.en:
        duplicates.append(pair.Index)
    if pair.en == pair.ar:
        duplicates.append(pair.Index)
    if pair.ar in pair.en:
        duplicates.append(pair.Index)
    if pair.en in pair.ar:
        duplicates.append(pair.Index)


In [17]:
duplicates = list(set(duplicates))

In [18]:
len(duplicates)

10393

In [19]:
df = df.drop(duplicates)
df.shape

(988979, 2)

In [20]:
dedup = df.drop_duplicates(subset=['ar', 'en'])

In [23]:
dedup.to_parquet('ar-en.parquet', index=False)

In [24]:
dedup.head(5)

,ar,en
0,إلى ربها ناظرة: أي منتظرة لرحمته ودخول جنته.,He is sometimes seen having arguments with his...
1,في البداية حققوا بعض المكاسب بما في ذلك الاستي...,"Initially, they made some gains, including cap..."
2,الأدميرال هالسي (1944) يرتدي خوذة الألياف الأم...,Admiral Halsey (1944) wearing an American fibe...
4,"عن ""رب العالمين"" سبحانه وتعالى.",Opsaridium ubangiense.
5,النّعيم : الذي ألهاكم عن طاعة ربكم .,Thus hath the Lord your God commanded you.
